# Exploratory Data Analysis Car Advertisement

In [10]:
# Loading all the libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats as sp
import streamlit as st
import plotly_express as pl

In [11]:
df = pd.read_csv('vehicles_us.csv')

In [12]:
df.info()
display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
5,14990,2014.0,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15
6,12990,2015.0,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,NaN,2018-12-27,73
7,15990,2013.0,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68
8,11500,2012.0,kia sorento,excellent,4.0,gas,104174.0,automatic,SUV,NaN,1.0,2018-07-16,19
9,9200,2008.0,honda pilot,excellent,NaN,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17


In [13]:
#Clean data
df['date_posted'] = pd.to_datetime(df['date_posted'])
df['model_year'] = df['model_year'].fillna(9999).astype(int)
df['odometer'] = df['odometer'].fillna(999999).astype(int)
df['paint_color'] = df['paint_color'].astype('str', errors='ignore')
#drop unessary columns and rows
df = df.drop(['cylinders', 'fuel', 'transmission', 'is_4wd'], axis=1)
df = df.drop_duplicates()
# drop duplicate values
df['type'] = df['type'].replace({'pickup': 'truck'})
df['type'] = df['type'].replace({'offroad': 'SUV'})
df_clean =df

In [14]:
display(df_clean.info())
display(df_clean.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   price        51525 non-null  int64         
 1   model_year   51525 non-null  int64         
 2   model        51525 non-null  object        
 3   condition    51525 non-null  object        
 4   odometer     51525 non-null  int64         
 5   type         51525 non-null  object        
 6   paint_color  51525 non-null  object        
 7   date_posted  51525 non-null  datetime64[ns]
 8   days_listed  51525 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 3.5+ MB


None

,price,model_year,model,condition,odometer,type,paint_color,date_posted,days_listed
0,9400,2011,bmw x5,good,145000,SUV,nan,2018-06-23,19
1,25500,9999,ford f-150,good,88705,truck,white,2018-10-19,50
2,5500,2013,hyundai sonata,like new,110000,sedan,red,2019-02-07,79
3,1500,2003,ford f-150,fair,999999,truck,nan,2019-03-22,9
4,14900,2017,chrysler 200,excellent,80903,sedan,black,2019-04-02,28
5,14990,2014,chrysler 300,excellent,57954,sedan,black,2018-06-20,15
6,12990,2015,toyota camry,excellent,79212,sedan,white,2018-12-27,73
7,15990,2013,honda pilot,excellent,109473,SUV,black,2019-01-07,68
8,11500,2012,kia sorento,excellent,104174,SUV,nan,2018-07-16,19
9,9200,2008,honda pilot,excellent,147191,SUV,blue,2019-02-15,17


## Modeling Data


In [21]:
#average price by vehicle type
avg_type = df.groupby('type')['price'].mean().reset_index()
avg_type = avg_type.sort_values(by='price', ascending=True)

# average price by model year
avg_year = df.groupby('model_year')['price'].mean().reset_index()

# removed extreme values for year to prevent data being skewed
avg_year = avg_year[(avg_year['model_year'] <= 2020) & (avg_year['model_year'] >= 1995)]

# average price by condition 
avg_condition = df.groupby('condition')['price'].mean().reset_index()
avg_condition = avg_condition.sort_values(by='price', ascending=True)
# Create bar charts
fig1 = pl.bar(avg_type, x='type', y='price', title='Average Price by Vehicle Type',  color_discrete_sequence=['#33ccff'], labels={'type': 'Vehicle Type', 'price': 'Average Price'})
fig1.update_traces(marker_line_color='black', marker_line_width=1.5)

fig2 = pl.bar(avg_year, x='model_year', y='price', title='Average Price by Model Year',  color_discrete_sequence=['#8585e0'], labels={'model_year': 'Model Year', 'price': 'Average Price'})
fig2.update_traces(marker_line_color='black', marker_line_width=1.5)

fig3 = pl.bar(avg_condition, x='condition', y='price', title='Average Price by Condition',  color_discrete_sequence=['#b300b3'], labels={'model_year': 'Model Year', 'price': 'Average Price'})
fig3.update_traces(marker_line_color='black', marker_line_width=1.5)


fig1.show()
fig2.show()
fig3.show()

In [7]:
# average days listed by type
avg_days_listed_type = df.groupby('type')['days_listed'].mean().reset_index()
avg_days_listed_type = avg_days_listed_type[avg_days_listed_type['type'] != 'other']
# average days listed by model year
avg_year_days = df.groupby('model_year')['days_listed'].mean().reset_index()
avg_year_days = avg_year_days.sort_values(by='days_listed', ascending=False)
avg_days_listed_year = avg_year_days.groupby('model_year')['days_listed'].mean().reset_index()
avg_days_listed_year = avg_days_listed_year[(avg_days_listed_year['model_year'] <= 2020) & (avg_days_listed_year['model_year'] >= 1995)]

# average days listed by color
avg_days_listed_color = df.groupby('paint_color')['days_listed'].mean().reset_index()

# average days listed by condition
avg_days_listed_condition = df.groupby('condition')['days_listed'].mean().reset_index()

# create scatter plots
fig1 = pl.scatter(avg_days_listed_type, x='type', y='days_listed', title='Average Days Listed by Vehicle Type', labels={'days_listed': 'Days Listed', 'type': 'Vehicle Type'})
fig1.update_traces(marker=dict(size=30, color='#33ccff'))
fig1.update_traces(marker_line_color='black', marker_line_width=2.5)

fig2 = pl.scatter(avg_days_listed_year, x='model_year', y='days_listed', title='Average Days Listed by Model Year', labels={'days_listed': 'Days Listed', 'model_year': 'Model Year'})
fig2.update_traces(marker=dict(size=30, color='#8585e0'))
fig2.update_traces(marker_line_color='black', marker_line_width=2.5)

fig3 = pl.scatter(avg_days_listed_color, x='paint_color', y='days_listed', title='Average Days Listed by Paint Color', labels={'days_listed': 'Days Listed', 'paint_color': 'Paint Color'})
fig3.update_traces(marker=dict(size=30, color='#ff0066', line=dict(color='black', width=2.5)))

# Create scatter plot for average days listed by condition
fig4 = pl.scatter(avg_days_listed_condition, x='condition', y='days_listed', title='Average Days Listed by Condition', labels={'days_listed': 'Days Listed', 'condition': 'Vehicle Condition'})
fig4.update_traces(marker=dict(size=30, color= '#ffff00', line=dict(color='black', width=2.5)))

fig4.show()
fig3.show()
fig1.show()
fig2.show()



In [8]:
# Filter average days listed by prices, for vehicles less than 60 days
avg_days_listed_price = df.groupby('price')['days_listed'].mean().reset_index()
avg_days_listed_price = avg_days_listed_price[avg_days_listed_price['days_listed'] <= 60]

# distribution of vehicles price more than 20,000 by days, and less 60 days
avg_days_listed_price_above_20000 = avg_days_listed_price[avg_days_listed_price['price'] >= 20000]

# distribution of vehicles price less than 20,000 by days, and less 60 days
avg_days_listed_price_below_20000 = avg_days_listed_price[avg_days_listed_price['price'] < 20000]

# create histograms
fig1 = pl.histogram(avg_days_listed_price_above_20000, x='days_listed', nbins=30, title='Average Days Vehicles Listed Above 20,000 Dollars', labels={'days_listed': 'Average Days Listed'}, color_discrete_sequence=['#33ccff'])

fig1.update_traces(marker_line_color='black', marker_line_width=1.5)
fig1.update_yaxes(range=[0, 160])

fig2 = pl.histogram(avg_days_listed_price_below_20000, x='days_listed', nbins=30, title='Average Days Vehicles Listed Below 20,000 Dollars', labels={'days_listed': 'Average Days Listed'}, color_discrete_sequence=['#8585e0'])

fig2.update_traces(marker_line_color='black', marker_line_width=1.5)
fig1.update_yaxes(range=[0, 160])

fig2.show()
fig1.show()